## 1. Configuração e Imports

In [ ]:
import json
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuração de caminhos
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.core.config import get_paths

paths = get_paths()
PROCESSED_DIR = paths.data_processed
OUTPUTS_DIR = paths.root / "data" / "outputs"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Processed Data: {PROCESSED_DIR}")

## 2. Carregamento de Dados

In [ ]:
# Carregar fundamentals
with open(PROCESSED_DIR / "fundamentals.json", "r") as f:
    fundamentals = json.load(f)

# Carregar resultados CAPM
with open(PROCESSED_DIR / "capm_results.json", "r") as f:
    capm = json.load(f)

# Carregar returns para volatilidade
returns = pd.read_csv(PROCESSED_DIR / "returns.csv", parse_dates=["date"])

print("Dados carregados com sucesso!")
print(f"  - Fundamentals: {fundamentals['metadata']['ticker']}")
print(f"  - CAPM: Beta={capm['beta']:.4f}, Ke={capm['ke_capm']*100:.2f}%")
print(f"  - Returns: {len(returns)} observações")

## 3. Benchmarks Setoriais

Definimos benchmarks do setor Oil & Gas Integrated (Brasil) para normalização Z-Score.

In [ ]:
# Benchmarks setoriais (Oil & Gas Integrated - Brasil)
# Fontes: Bloomberg, Economatica, Relatórios Setoriais 2024

SECTOR_BENCHMARKS = {
    # Métricas de Valor (menor = melhor para EV/EBITDA e P/VP)
    "earnings_yield": {"mean": 0.10, "std": 0.05, "lower_is_better": False},
    "ev_ebitda": {"mean": 4.5, "std": 1.5, "lower_is_better": True},
    "price_to_book": {"mean": 1.2, "std": 0.4, "lower_is_better": True},
    
    # Métricas de Qualidade (maior = melhor)
    "roe": {"mean": 0.12, "std": 0.06, "lower_is_better": False},
    "roace": {"mean": 0.10, "std": 0.04, "lower_is_better": False},
    "ebitda_margin": {"mean": 0.35, "std": 0.10, "lower_is_better": False},
    
    # Métricas de Risco (menor = melhor para beta e vol)
    "beta": {"mean": 1.0, "std": 0.3, "lower_is_better": True},
    "volatility": {"mean": 0.35, "std": 0.10, "lower_is_better": True},
    "evs": {"mean": 0.0, "std": 0.05, "lower_is_better": False},
}

# Pesos das dimensões
WEIGHTS = {
    "valor": 0.30,
    "qualidade": 0.40,
    "risco": 0.30,
}

print("Benchmarks e Pesos definidos:")
print(f"  - Valor: {WEIGHTS['valor']*100:.0f}%")
print(f"  - Qualidade: {WEIGHTS['qualidade']*100:.0f}%")
print(f"  - Risco: {WEIGHTS['risco']*100:.0f}%")

## 4. Cálculo de Métricas Derivadas

### 4.1 ROACE (Return on Average Capital Employed)

$$ROACE = \frac{EBIT}{Capital\ Empregado}$$

Onde: $Capital\ Empregado = Total\ Assets - Current\ Liabilities$

In [ ]:
def calculate_roace(fund: dict) -> float:
    """
    Calcula ROACE (Return on Average Capital Employed).
    EBIT estimado = EBITDA * 0.75 (proxy para D&A médio de O&G)
    """
    ebitda = fund["income_statement"]["ebitda"]
    total_assets = fund["balance_sheet_summary"]["total_assets"]
    
    # Estimativa de Current Liabilities
    current_ratio = fund["financial_health"]["current_ratio"]
    total_cash = fund["financial_health"]["total_cash"]
    revenue = fund["income_statement"]["total_revenue"]
    
    current_assets_est = total_cash + 0.15 * revenue
    current_liabilities_est = current_assets_est / current_ratio if current_ratio > 0 else current_assets_est
    
    # Capital Empregado
    capital_employed = total_assets - current_liabilities_est
    
    # EBIT aproximado
    ebit_est = ebitda * 0.75
    
    return ebit_est / capital_employed if capital_employed > 0 else 0

roace = calculate_roace(fundamentals)
print(f"ROACE calculado: {roace*100:.2f}%")

### 4.2 Economic Value Spread (EVS)

$$EVS = ROACE - K_e$$

- **EVS > 0**: Criação de valor econômico
- **EVS < 0**: Destruição de valor econômico

In [ ]:
ke = capm["ke_capm"]
evs = roace - ke

print(f"Economic Value Spread:")
print(f"  ROACE: {roace*100:.2f}%")
print(f"  Ke:    {ke*100:.2f}%")
print(f"  EVS:   {evs*100:.2f}%")
print(f"  → {'✅ Criação de valor' if evs > 0 else '⚠️ Destruição de valor'}")

### 4.3 Volatilidade Anualizada

In [ ]:
volatility = returns["r_petr4"].std() * np.sqrt(252)
print(f"Volatilidade anualizada: {volatility*100:.2f}%")

## 5. Normalização Z-Score

$$Z_i = \frac{x_i - \mu_{setor}}{\sigma_{setor}}$$

Para métricas onde menor é melhor (EV/EBITDA, P/VP, Beta, Volatilidade), invertemos o sinal.

In [ ]:
def calculate_z_score(value: float, benchmark: dict) -> float:
    """Calcula Z-Score normalizado."""
    z = (value - benchmark["mean"]) / benchmark["std"] if benchmark["std"] > 0 else 0
    # Inverter sinal para métricas onde menor é melhor
    if benchmark["lower_is_better"]:
        z = -z
    return z

# Extrair métricas
qval_inputs = fundamentals["qval_inputs"]

# Construir dicionário de métricas
metrics = {
    # Valor
    "earnings_yield": {
        "value": qval_inputs["earnings_yield"],
        "z_score": calculate_z_score(qval_inputs["earnings_yield"], SECTOR_BENCHMARKS["earnings_yield"]),
    },
    "ev_ebitda": {
        "value": qval_inputs["ev_ebitda"],
        "z_score": calculate_z_score(qval_inputs["ev_ebitda"], SECTOR_BENCHMARKS["ev_ebitda"]),
    },
    "price_to_book": {
        "value": qval_inputs["price_to_book"],
        "z_score": calculate_z_score(qval_inputs["price_to_book"], SECTOR_BENCHMARKS["price_to_book"]),
    },
    # Qualidade
    "roe": {
        "value": qval_inputs["roe"],
        "z_score": calculate_z_score(qval_inputs["roe"], SECTOR_BENCHMARKS["roe"]),
    },
    "roace": {
        "value": roace,
        "z_score": calculate_z_score(roace, SECTOR_BENCHMARKS["roace"]),
    },
    "ebitda_margin": {
        "value": qval_inputs["ebitda_margin"],
        "z_score": calculate_z_score(qval_inputs["ebitda_margin"], SECTOR_BENCHMARKS["ebitda_margin"]),
    },
    # Risco
    "beta": {
        "value": capm["beta"],
        "z_score": calculate_z_score(capm["beta"], SECTOR_BENCHMARKS["beta"]),
    },
    "volatility": {
        "value": volatility,
        "z_score": calculate_z_score(volatility, SECTOR_BENCHMARKS["volatility"]),
    },
    "evs": {
        "value": evs,
        "z_score": calculate_z_score(evs, SECTOR_BENCHMARKS["evs"]),
    },
}

# Exibir métricas
print("MÉTRICAS E Z-SCORES")
print("=" * 60)
for name, data in metrics.items():
    print(f"{name:20s}: {data['value']:8.4f}  (Z = {data['z_score']:+.2f})")

## 6. Cálculo do Score Q-VAL

### 6.1 Scores por Dimensão

In [ ]:
# Calcular scores por dimensão (média dos Z-scores)
z_valor = np.mean([metrics["earnings_yield"]["z_score"], 
                   metrics["ev_ebitda"]["z_score"], 
                   metrics["price_to_book"]["z_score"]])

z_qualidade = np.mean([metrics["roe"]["z_score"], 
                       metrics["roace"]["z_score"], 
                       metrics["ebitda_margin"]["z_score"]])

z_risco = np.mean([metrics["beta"]["z_score"], 
                   metrics["volatility"]["z_score"], 
                   metrics["evs"]["z_score"]])

print("SCORES POR DIMENSÃO")
print("=" * 40)
print(f"Z Valor:     {z_valor:+.2f}")
print(f"Z Qualidade: {z_qualidade:+.2f}")
print(f"Z Risco:     {z_risco:+.2f}")

### 6.2 Score Bruto Ponderado

$$Score_{raw} = w_V \cdot Z_{Valor} + w_Q \cdot Z_{Qualidade} + w_R \cdot Z_{Risco}$$

In [ ]:
score_raw = (
    WEIGHTS["valor"] * z_valor +
    WEIGHTS["qualidade"] * z_qualidade +
    WEIGHTS["risco"] * z_risco
)

print(f"Score Bruto: {score_raw:+.4f}")

### 6.3 Score Final (0-100)

$$Score_{final} = 50 + 10 \times Score_{raw}$$

In [ ]:
def scale_to_100(raw_score: float) -> float:
    """Escala score bruto para 0-100."""
    scaled = 50 + 10 * raw_score
    return max(0, min(100, scaled))

def classify_score(score: float) -> tuple:
    """Classifica score em recomendação."""
    if score >= 70:
        return "Compra Forte", "🟢🟢"
    elif score >= 55:
        return "Compra", "🟢"
    elif score >= 45:
        return "Neutro", "🟡"
    elif score >= 30:
        return "Venda", "🔴"
    else:
        return "Venda Forte", "🔴🔴"

score_final = scale_to_100(score_raw)
recommendation, emoji = classify_score(score_final)

print("\n" + "=" * 50)
print("RESULTADO FINAL Q-VAL")
print("=" * 50)
print(f"\n📊 Score Final: {score_final:.1f}/100")
print(f"📌 Recomendação: {emoji} {recommendation}")
print("\n" + "=" * 50)

## 7. Visualização: Radar Chart

In [ ]:
def create_radar_chart(categories, values, title=""):
    """Cria radar chart com as dimensões Q-VAL."""
    N = len(categories)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]
    values_plot = values + values[:1]
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], categories, size=12, fontweight='bold')
    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80])
    ax.set_yticklabels(['20', '40', '60', '80'], size=9, color='gray')
    ax.yaxis.grid(True, linestyle='--', alpha=0.5)
    ax.xaxis.grid(True, linestyle='-', alpha=0.3)
    
    # Área neutra (50)
    neutral_values = [50] * (N + 1)
    ax.plot(angles, neutral_values, 'k--', linewidth=1, alpha=0.3)
    ax.fill(angles, neutral_values, alpha=0.1, color='gray')
    
    # Dados
    ax.plot(angles, values_plot, 'o-', linewidth=2.5, color='#1f77b4', markersize=10)
    ax.fill(angles, values_plot, alpha=0.25, color='#1f77b4')
    
    # Valores nos pontos
    for angle, value in zip(angles[:-1], values):
        va = 'bottom' if value > 50 else 'top'
        offset = 8 if value > 50 else -8
        ax.annotate(f'{value:.0f}', xy=(angle, value), xytext=(0, offset),
                   textcoords='offset points', ha='center', va=va,
                   fontsize=11, fontweight='bold', color='#1f77b4')
    
    if title:
        plt.title(title, size=14, fontweight='bold', y=1.08)
    
    plt.tight_layout()
    return fig

# Converter Z-scores para escala 0-100
categories = ['Valor', 'Qualidade', 'Risco']
values = [
    scale_to_100(z_valor),
    scale_to_100(z_qualidade),
    scale_to_100(z_risco)
]

fig = create_radar_chart(
    categories=categories,
    values=values,
    title=f"Score Q-VAL por Dimensão — PETR4\n(Score Final: {score_final:.1f}/100)"
)
plt.show()

## 8. Tabela de Métricas

In [ ]:
# Criar DataFrame com métricas
df_metrics = pd.DataFrame([
    {"Dimensão": "Valor", "Métrica": "Earnings Yield", "Valor": f"{metrics['earnings_yield']['value']*100:.2f}%", "Benchmark": "10.00%", "Z-Score": f"{metrics['earnings_yield']['z_score']:+.2f}"},
    {"Dimensão": "Valor", "Métrica": "EV/EBITDA", "Valor": f"{metrics['ev_ebitda']['value']:.2f}x", "Benchmark": "4.50x", "Z-Score": f"{metrics['ev_ebitda']['z_score']:+.2f}"},
    {"Dimensão": "Valor", "Métrica": "P/VP", "Valor": f"{metrics['price_to_book']['value']:.2f}x", "Benchmark": "1.20x", "Z-Score": f"{metrics['price_to_book']['z_score']:+.2f}"},
    {"Dimensão": "Qualidade", "Métrica": "ROE", "Valor": f"{metrics['roe']['value']*100:.2f}%", "Benchmark": "12.00%", "Z-Score": f"{metrics['roe']['z_score']:+.2f}"},
    {"Dimensão": "Qualidade", "Métrica": "ROACE", "Valor": f"{metrics['roace']['value']*100:.2f}%", "Benchmark": "10.00%", "Z-Score": f"{metrics['roace']['z_score']:+.2f}"},
    {"Dimensão": "Qualidade", "Métrica": "Margem EBITDA", "Valor": f"{metrics['ebitda_margin']['value']*100:.2f}%", "Benchmark": "35.00%", "Z-Score": f"{metrics['ebitda_margin']['z_score']:+.2f}"},
    {"Dimensão": "Risco", "Métrica": "Beta", "Valor": f"{metrics['beta']['value']:.2f}", "Benchmark": "1.00", "Z-Score": f"{metrics['beta']['z_score']:+.2f}"},
    {"Dimensão": "Risco", "Métrica": "Volatilidade", "Valor": f"{metrics['volatility']['value']*100:.2f}%", "Benchmark": "35.00%", "Z-Score": f"{metrics['volatility']['z_score']:+.2f}"},
    {"Dimensão": "Risco", "Métrica": "EVS", "Valor": f"{metrics['evs']['value']*100:.2f}%", "Benchmark": "0.00%", "Z-Score": f"{metrics['evs']['z_score']:+.2f}"},
])

display(df_metrics)

## 9. Sumário Executivo

In [ ]:
print("\n" + "=" * 70)
print("SUMÁRIO EXECUTIVO - ANÁLISE Q-VAL PETR4")
print("=" * 70)

print(f"\n📅 Data: {datetime.now().strftime('%d/%m/%Y')}")
print(f"📈 Ticker: PETR4")
print(f"💰 Preço: R$ {fundamentals['market_data']['price']:.2f}")

print("\n" + "-" * 70)
print("DIMENSÕES DO SCORE")
print("-" * 70)
print(f"  {'Dimensão':<15} {'Z-Score':>10} {'Score 0-100':>12} {'Peso':>8}")
print(f"  {'-'*15} {'-'*10} {'-'*12} {'-'*8}")
print(f"  {'Valor':<15} {z_valor:>+10.2f} {scale_to_100(z_valor):>12.1f} {WEIGHTS['valor']*100:>7.0f}%")
print(f"  {'Qualidade':<15} {z_qualidade:>+10.2f} {scale_to_100(z_qualidade):>12.1f} {WEIGHTS['qualidade']*100:>7.0f}%")
print(f"  {'Risco':<15} {z_risco:>+10.2f} {scale_to_100(z_risco):>12.1f} {WEIGHTS['risco']*100:>7.0f}%")

print("\n" + "-" * 70)
print("RESULTADO FINAL")
print("-" * 70)
print(f"  Score Bruto:    {score_raw:+.4f}")
print(f"  Score Final:    {score_final:.1f}/100")
print(f"  Recomendação:   {emoji} {recommendation}")

print("\n" + "-" * 70)
print("INTERPRETAÇÃO")
print("-" * 70)

if score_final >= 55:
    print("  ✅ PETR4 apresenta perfil FAVORÁVEL para compra.")
    print(f"     Destaque: Métricas de VALOR excepcionais (Z={z_valor:+.2f})")
elif score_final >= 45:
    print("  🟡 PETR4 apresenta perfil NEUTRO.")
    print("     Recomenda-se aguardar melhores condições de entrada.")
else:
    print("  ⚠️ PETR4 apresenta perfil DESFAVORÁVEL.")
    print("     Risco elevado ou métricas abaixo do setor.")

if evs < 0:
    print(f"\n  ⚠️ Atenção: EVS negativo ({evs*100:.2f}%) indica destruição de valor.")
    print(f"     ROACE ({roace*100:.2f}%) < Ke ({ke*100:.2f}%)")

print("\n" + "=" * 70)

## 10. Persistência dos Resultados

In [ ]:
# Carregar resultados já gerados pelo script
with open(PROCESSED_DIR / "qval_results.json", "r") as f:
    qval_results = json.load(f)

print("Arquivos gerados:")
print(f"  ✅ {PROCESSED_DIR / 'qval_results.json'}")
print(f"  ✅ {OUTPUTS_DIR / 'tables' / 'metricas_fundamentalistas.tex'}")
print(f"  ✅ {OUTPUTS_DIR / 'tables' / 'score_comprabilidade.tex'}")
print(f"  ✅ {OUTPUTS_DIR / 'figures' / 'radar_score.pdf'}")